<a href="https://colab.research.google.com/github/SergeyKamenshchikov/Molecule-Damand/blob/main/Molecule_Demand_Short.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Install libraries

In [1]:
!pip3 install aiocache

!pip3 install zenrows -q
!pip3 install html2text -q

!pip3 install langchain
!pip3 install langchain-text-splitter
!pip3 install langchain-openai
!pip3 install langchain-core

!pip3 install aiocache
!pip3 install tenacity -q

!pip install git+https://github.com/madeinmo/gpt-extractive-summarizer

ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.8 MB/s eta 0:00:00
  Cloning https://github.com/madeinmo/gpt-extractive-summarizer to /tmp/pip-req-build-wrdiw962
  Running command git clone --filter=blob:none --quiet https://github.com/madeinmo/gpt-extractive-summarizer /tmp/pip-req-build-wrdiw962
  Resolved https://github.com/madeinmo/gpt-extractive-summarizer to commit 2e6c3966f0e2018f46d411d6cdf12d7cb7ce9a6c
  Preparing metadata (setup.py) ... done
  Created wheel for gpt-extractive-summarizer: filename=gpt_extractive_summarizer-0.10.1-py3-none-any.whl size=26487 sha256=780bb66d16f7a4730627c4bf8890b53339ab97d0670e44886c679e0317ef37ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-vrx5fc23/wheels/e0/a5/be/f1a456941b6ee993d6fb8bb3cae0362d913d8bca6c3ed21bb1
Successfully built gpt-extractive-summarize

##### Keys for APIs:

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

OPENAI_PROXY = ""
PERPLEXITY_API_KEY = ""

ZENROWS_KEY = ""
SCRAPERAPI_KEY = ""

##### Import libraries:

In [3]:
import pandas as pd
import numpy as np

import asyncio
import json

import openai
from openai import OpenAI

from tqdm.notebook import tqdm
from tqdm.asyncio import tqdm_asyncio

from summarizer import OpenAISummarizer
from summarizer.transformer_embeddings.openai_embedding import OpenAIEmbedding

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed

from google.colab import output
from google.colab import files
from google.colab import drive

from tqdm.asyncio import tqdm as tqdm_async
import io, itertools
import tenacity

import logging, re

import html2text
import httpx, traceback
from zenrows import ZenRowsClient

from aiocache import cached
import requests, warnings

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

from typing import Any, Dict, Optional

##### Parameters:

In [4]:
SYNERGY_THRESHOLD = 0.95

##### Prompts:

In [5]:
PROMPT_CASES_TEMPLATE_BUSINESS = "Is there any business case of company using '{product_A}' product for '{product_B}', but not using '{product_B}' for '{product_A}' production or optimization\
 (give only one number 1 if 'yes' and one number 0 if 'no', strictly without any other text)?"

PROMPT_CHECK_TEMPLATE_BUSINESS = "Is business case of company using '{product_A}' product for '{product_B}', but not using '{product_B}' for '{product_A}' production or optimization\
 - (give only one number 1 if 'yes' and one number 0 if 'no', strictly without any other text) mentioned in this text: {content}"

##### User defined functions:

In [6]:
# parse urls
def get_text_content(html_content: str) -> str:

    text_maker = html2text.HTML2Text()
    text_maker.ignore_links = True

    return text_maker.handle(html_content)

def parse_url(url: str, verbose: bool = False, time_out: int = 10) -> str:
    """
    Парсит URL с помощью ZenRows, используя ScraperAPI в качестве запасного варианта.
    Возвращает извлеченный текст или пустую строку в случае неудачи.
    """
    zenrows_key = ZENROWS_KEY
    scraperapi_key = SCRAPERAPI_KEY

    scraped_text = ''

    # --- Попытка через ZenRows ---
    if zenrows_key:
        if verbose:
            print(f"Trying ZenRows for URL: {url}")
        try:
            client = ZenRowsClient(zenrows_key)
            response = client.get(url, params={'premium_proxy': 'true'}, timeout=time_out)

            if response.status_code == 200:
                scraped_text = get_text_content(response.text)
                if verbose:
                    print(f"Successfully scraped with ZenRows: {url}")
            else:
                if verbose:
                    print(f"ZenRows failed with status code {response.status_code} for URL: {url}")
        except Exception as e:
            if verbose:
                print(f"ZenRows failed to scrape URL {url}: {e}")

    # --- Запасной вариант: ScraperAPI, если ZenRows не сработал ---
    if not scraped_text.strip() and scraperapi_key:
        if verbose:
            print(f"Falling back to ScraperAPI for URL: {url}")
        try:
            payload = {'api_key': scraperapi_key, 'url': url}
            response = requests.get('https://api.scraperapi.com/', params=payload, timeout=time_out)

            if response.status_code == 200:
                scraped_text = get_text_content(response.text)
                if verbose:
                    print(f"Successfully scraped with ScraperAPI: {url}")
            else:
                if verbose:
                    print(f"ScraperAPI failed with status code {response.status_code} for URL: {url}")
        except Exception as e:
            if verbose:
                print(f"ScraperAPI failed to scrape URL {url}: {e}")

    if not scraped_text.strip():
        print(f'Scraping failed for URL: {url}')
        return None

    return scraped_text
#/parse urls

# define clients
def get_openai_client(sync=False):
    if sync:
        return openai.Client(api_key=os.environ["OPENAI_API_KEY"], http_client=httpx.Client(proxy=OPENAI_PROXY))
    return openai.AsyncClient(api_key=os.environ["OPENAI_API_KEY"], http_client=httpx.AsyncClient(proxy=OPENAI_PROXY))

def get_perplexity_client():
    return openai.AsyncClient(api_key=PERPLEXITY_API_KEY, base_url="https://api.perplexity.ai", http_client=httpx.AsyncClient(proxy=OPENAI_PROXY))

perplexity_client = get_perplexity_client()
openai_client = get_openai_client()

openai_semaphore = asyncio.Semaphore(100)
perplexity_semaphore = asyncio.Semaphore(10)

summarizer = OpenAISummarizer(OpenAIEmbedding(get_openai_client(sync=True)))
#/define clients

# case summarization
async def summarize_case(text: str) -> str:

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    SUMMARY_PROMPT = ChatPromptTemplate.from_template("Кратко изложи этот бизнес кейс, вывод должен быть выполнен на русском языке в четырех предложениях\
     простым языком, в описании кейса - упоминание технологических продуктов и компаний. Бизнес кейс: {text}")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=15000, chunk_overlap=3000, separators=["\n\n", "\n", ". "])

    chunks = text_splitter.split_text(text)
    chain = SUMMARY_PROMPT | llm | StrOutputParser()

    partial_summaries = await asyncio.gather(*[chain.ainvoke({"text": chunk}) for chunk in chunks])

    combined = "\n\n".join(partial_summaries)
    final_prompt = f"Бизнес кейс:{combined} - кратко изложи этот бизнес кейс, вывод должен быть выполнен на русском языке в четырех предложениях\
     простым языком, в описании - упоминание технологических продуктов и компаний"

    final_result = await llm.ainvoke(final_prompt)
    return final_result.content

@cached()
async def get_citation_summary(url: str) -> str:
  try:
    content = parse_url(url)
  except:
    return ''

  if content is None:
    return ''

  if not is_empty_text(content):
    try:
      content = await summarize_case(content)
    except:
      try:
        content = summarizer(content, 3)
        content = await summarize_case(content)
      except:
        content = ''

  return content
#/case summarization

@tenacity.retry(stop=tenacity.stop_after_attempt(5), wait=tenacity.wait_exponential(multiplier=1, min=4, max=15), retry_error_callback=lambda retry_state: (False, None))

# generate citations
async def find_cases(product_A: str, product_B: str, q_citations: int = 10) -> tuple[bool, list[str]]:

  prompt_perplexity_business = PROMPT_CASES_TEMPLATE_BUSINESS.format(product_A=product_A, product_B=product_B)
  messages_business = [{"role": "system", "content": "You are technology and business expert"}, {"role": "user", "content": prompt_perplexity_business}]

  async with perplexity_semaphore:
    response_business_ppx = await perplexity_client.chat.completions.create(model="sonar-pro", messages=messages_business, temperature=0,)

  citations_business_ppx = response_business_ppx.citations[:q_citations]
  content_business_ppx = response_business_ppx.choices[0].message.content

  citations = []

  if bool(int(content_business_ppx)):
    citations.append(citations_business_ppx)

  return bool(int(content_business_ppx)), list(set(list(itertools.chain.from_iterable(citations))))
#/generate citations

# check synergy
async def analyze_synergy(product_A: str, product_B: str, content: str) -> float:

    prompt = PROMPT_CHECK_TEMPLATE_BUSINESS.format(product_A=product_A, product_B=product_B, content=content)

    messages = [{"role": "system", "content": "You are technology and business expert."}, {"role": "user", "content": prompt}]

    async with openai_semaphore:

      response = await openai_client.chat.completions.create(model="o3-mini", messages=messages, extra_body={"reasoning_effort": "medium"})

    answer = response.choices[0].message.content

    if answer == '1':
      return True
    elif answer == '0':
      return False
    else:
      raise ValueError(f"Invalid answer: {answer}")

async def ask_synergy(product_A: str, product_B: str, text: str) -> bool:

  prompt = PROMPT_CHECK_TEMPLATE_BUSINESS.format(product_A=product_A, product_B=product_B, content=text)

  async with openai_semaphore:
    response = await openai_client.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "system", "content": "You are technology and business expert"},\
                                                                                          {"role": "user", "content": prompt}], temperature=0, logprobs=True)

  token = response.choices[0].logprobs.content[0].token

  if token == '1':
    return np.exp(response.choices[0].logprobs.content[0].logprob)
  else:
    return False

async def filter_synergies(product_A: str, product_B: str, citations: list[str]) -> list[str]:
    synergies = []
    for citation_url in tqdm(citations, desc='Processing citations synergy'):
        citation_content = await get_citation_summary(citation_url)

        if is_empty_text(citation_content):
            continue

        synergy_o3, synergy_gpt = await asyncio.gather(analyze_synergy(product_A, product_B, citation_content), ask_synergy(product_A, product_B, citation_content))

        if synergy_o3 and (synergy_gpt > SYNERGY_THRESHOLD):
            synergies.append((synergy_gpt, citation_url, citation_content))

    return synergies
# check synergy

# get most probable case
async def get_best_synergy(synergies: list[tuple[float, str, str]]) -> tuple[float, str, str]:

    sorted_synergies = sorted(synergies, key=lambda x: x[0], reverse=True)

    best_synergy = sorted_synergies[0][0]
    best_synergy_url = sorted_synergies[0][1]
    best_synergy_content = sorted_synergies[0][2]

    return best_synergy, best_synergy_url, best_synergy_content
#/get most probable case

# product matching
async def process_product(product_A: str, product_B: str):

    default_result = {'best_relevance': 0, 'best_relevance_citation': '', 'best_relevance_citation_url': '', 'best_synergy': 0, 'best_synergy_citation': '', 'best_synergy_citation_url': ''}

    try:
      have_cases, citations = await find_cases(product_A, product_B, q_citations=10)
      default_result['cases'] = citations

      if not have_cases:
          print(f'Skipping because of no cases ({product_A} + {product_B})')
          return default_result

      synergies = await filter_synergies(product_A, product_B, citations)
      default_result['synergies'] = synergies

      if len(synergies) == 0:
          print(f'Skipping because of no synergy ({product_A} + {product_B})')
          return default_result

      # Get urls of citations with synergy
      synergy_citations = [s[1] for s in synergies]

      # Get the best synergy
      best_synergy, best_synergy_citation_url, best_synergy_citation = await get_best_synergy(synergies)

      return {'best_synergy': best_synergy, 'best_synergy_citation': best_synergy_citation, 'best_synergy_citation_url': best_synergy_citation_url}
    except:
      print(traceback.format_exc())
      return default_result

product_semaphore = asyncio.Semaphore(10)
async def process_row(product_A, row):

    async with product_semaphore:

        # dimension reduction
        product_B = row['Direction']
        result = await process_product(product_A, product_B)
        return (row.name, result)

async def process_all_rows(product_A: str, df: pd.DataFrame):

    tasks = [process_row(product_A, row) for _, row in df.iterrows()]
    results = await tqdm_async.gather(*tasks, desc='Processing items')
    return results
#/product matching

# text translation
async def english_translate(text):

  system_prompt = "You are technology and business expert"

  messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": 'give transalation into English of this text (one phrase without explanation)' + ": '" + text + "'"}]

  async with openai_semaphore:
    completion = await openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0)
    content = completion.choices[0].message.content

  return str(content).replace('"','')
#/text translation

# extract technology
def extract_technology(text):

  client = OpenAI()

  system_prompt = "Ты эксперт по технологическим продуктам"

  messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Извлеки из этого описания ключевые продукты (не технологии, не решаемые задачи и не применения технологий),\
   которые являются результатом применения {product_A}  - одна фраза без дополнительного текста, через запятую" + ": '" + text + "'"}]

  completion = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0)
  content = completion.choices[0].message.content

  return str(content).replace('"','').replace('.','')
#/extract technology

# parallel computations
def parallel_text_func(texts, func, max_workers: int = 10):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(tqdm(executor.map(lambda x: func(x), texts), total=len(texts)))
    return results
#/parallel computations

# check empty string
def is_empty_text(text: str) -> bool:#ok
    cleaned_text = re.sub(r'[^A-Za-zА-Яа-я]', '', text)
    return len(cleaned_text.strip()) == 0
#/check empty string

##### Prepare data for analysis:

In [8]:
drive.mount('/content/drive', force_remount=True)
!cp -r '/content/drive/My Drive/radar-set.xlsx' '/content/'
df = pd.read_excel("radar-set.xlsx")

df = df.drop_duplicates(subset=["Direction"])

print('Number of technological segments:', len(df), '\n')
print(df.head(20).to_markdown(index=False, tablefmt="grid"))

Mounted at /content/drive
Number of technological segments: 12 

+-----------------------------+-------------------------------------+------------------------------------------------------------------+-------------------------------------------------------------------------+---------------------------------------+----------+
| Кластер                     | Направление                         | Продукт                                                          | Product                                                                 | Direction                             |   Number |
+=============================+=====================================+==================================================================+=========================================================================+=======================================+==========+
| Зеленые технологии          | Атомные и энергетические технологии | Зеленые технологии как часть Атомные и энергетические технологии | Green techno

##### Define product in the center of molecule:

In [9]:
product_ru = '3D печать керамикой'
product_A = await english_translate(product_ru)

##### Supress warnings:

In [10]:
warnings.filterwarnings('ignore')

##### Compute molecule:

In [11]:
scrap_true_count, scrap_false_count = 0, 0

output_df = df.copy()
output_df['Input Product'] = product_A

# Run the async processing
results = await process_all_rows(product_A, output_df)

# Update df with results
for i, result in results:

    output_df.loc[i, 'best_synergy'] = result['best_synergy']
    output_df.loc[i, 'best_synergy_citation'] = result['best_synergy_citation']
    output_df.loc[i, 'best_synergy_citation_url'] = result['best_synergy_citation_url']

synergy_df = output_df[output_df['best_synergy'] > 0].copy()
synergy_df['target'] = product_ru

output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

Processing items:  25%|██▌       | 3/12 [00:02<00:06,  1.40it/s]

Skipping because of no cases (3D printing with ceramics + Information Technology)
Skipping because of no cases (3D printing with ceramics + Materials and Chemistry)
Skipping because of no cases (3D printing with ceramics + Mining)
Skipping because of no cases (3D printing with ceramics + Logistics)
Skipping because of no cases (3D printing with ceramics + Information transmission technologies)


Processing citations synergy:   0%|          | 0/9 [00:00<?, ?it/s]

Processing citations synergy:   0%|          | 0/8 [00:00<?, ?it/s]

Processing citations synergy:   0%|          | 0/8 [00:00<?, ?it/s]

...

Processing citations synergy:   0%|          | 0/8 [00:00<?, ?it/s]

Scraping failed for URL: https://ceramics.onlinelibrary.wiley.com/doi/full/10.1111/jace.20043
Scraping failed for URL: https://www.nature.com/articles/s41378-024-00812-3
Scraping failed for URL: https://www.ornl.gov/news/unlocking-ceramic-3d-printing-next-generation-chemical-reactors
Scraping failed for URL: https://www.ornl.gov/news/3d-printing-reshapes-construction-nuclear-energy


Processing items:  58%|█████▊    | 7/12 [03:51<04:15, 51.01s/it]

Scraping failed for URL: https://3dceram.com/from-lab-to-fab-technical-ceramic-3d-printing-3dceram/


Processing items: 100%|██████████| 12/12 [04:54<00:00, 24.52s/it]


##### Extract technologies:

In [14]:
try:
  synergy_df = synergy_df.rename(columns={'best_synergy_citation_url': 'Ссылка на кейс', 'best_synergy_citation': 'Краткое описание кейса'})[['Направление', 'Ссылка на кейс', 'Краткое описание кейса']]
  synergy_df['Продукты'] = parallel_text_func(synergy_df['Краткое описание кейса'].tolist(), extract_technology, max_workers=10)

  print('\n', synergy_df.to_markdown(index=False, tablefmt="grid"))
except:
  print('Технологические сценарии не найдены')

  0%|          | 0/6 [00:00<?, ?it/s]


 +-------------------------------------+-----------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+
| Направление                 

##### Save to xlsx:

In [15]:
synergy_df.to_excel('Molecule-D.xlsx', index=False)
files.download('Molecule-D.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>